## Script to convert match.json to a csv

In [5]:
import json
import pandas as pd
import numpy as np

# Load JSON data
with open('match_9/replay.json') as f:
    data = json.load(f)

# Prepare the DataFrame to collect all data
columns = ['time', 'player_id', 'kill', 'assist']
all_players_data = pd.DataFrame(columns=columns)

# Process each player's data
for player_key, player_data in data.items():
    if player_key.startswith('player_'):
        player_id = player_key.split('_')[1]
        # Convert kill and assist times to np.array and round them to nearest integer
        kill_times = np.round(np.array(player_data['killTimes'])).astype(int)
        assist_times = np.round(np.array(player_data['assistTimes'])).astype(int)
        
        # Determine the max time to cover all slots
        max_time = max(np.max(kill_times, initial=0), np.max(assist_times, initial=0))
        timeline = np.arange(0, max_time + 1)
        
        # Initialize kill and assist columns with zeros
        kills = np.zeros_like(timeline)
        assists = np.zeros_like(timeline)
        
        # Mark kills and assists at rounded times
        for kill_time in kill_times:
            if kill_time < len(kills):  # Ensure index is within the array bounds
                kills[kill_time] = 1
        for assist_time in assist_times:
            if assist_time < len(assists):  # Ensure index is within the array bounds
                assists[assist_time] = 1
        
        # Create a DataFrame for the player
        player_df = pd.DataFrame({
            'time': timeline,
            'player_id': player_id,
            'kill': kills,
            'assist': assists
        })
        
        # Append the player's data to the collective DataFrame
        all_players_data = pd.concat([all_players_data, player_df], ignore_index=True)

# Sort by player_id first, then by time
all_players_data = all_players_data.sort_values(by=['player_id', 'time']).reset_index(drop=True)

# Save to CSV
all_players_data.to_csv('match_9.csv', index=False)


## Script to merge sensor data

In [6]:
import pandas as pd
import numpy as np
import os

# Load the match data
match_df = pd.read_csv('match_9.csv')

# Ensure the 'time' column is of integer type
match_df['time'] = match_df['time'].astype(int)

# Define the path to the match_1 directory
match_dir = 'match_9'

# Placeholder DataFrame to hold combined data
combined_df = pd.DataFrame()

# Iterate over each player's directory in match_1
for player_folder in os.listdir(match_dir):
    if player_folder.startswith('player_'):
        player_id = player_folder.split('_')[1]
        folder_path = os.path.join(match_dir, player_folder)
        
        # Read and merge sensor data, ensuring the 'time' column is integer type
        for sensor_file in ['gsr.csv', 'heart_rate.csv', 'eeg_metrics.csv']:
            try:
                sensor_df = pd.read_csv(os.path.join(folder_path, sensor_file))
                # Ensure the 'time' column in sensor data is of integer type
                sensor_df['time'] = sensor_df['time'].astype(int)
                # Rename columns to include sensor type for clarity, skipping if it's 'time'
                sensor_df.columns = [f"{sensor_file.split('.')[0]}/{col}" if col != 'time' else col for col in sensor_df.columns]
                # Merge with the match data for this player
                match_df = match_df.merge(sensor_df, on='time', how='left')
            except FileNotFoundError:
                # If a file is not found, it skips without adding its data
                pass

# Now, filter the match_df for each player and append it to combined_df
for player_id in match_df['player_id'].unique():
    player_df = match_df[match_df['player_id'] == player_id]
    combined_df = pd.concat([combined_df, player_df], ignore_index=True)

# Sort by player_id first, then by time
combined_df.sort_values(by=['player_id', 'time'], inplace=True)

# Save the combined data to a new CSV file
combined_df.to_csv('augmented_match_data/match_9_augmented.csv', index=False)


/var/folders/z0/3x2n2w256hqc2mg2n4y7_0t00000gn/T/ipykernel_83505/4041771514.py:32: FutureWarning: Passing 'suffixes' which cause duplicate columns {'gsr/gsr_x'} in the result is deprecated and will raise a MergeError in a future version.
  match_df = match_df.merge(sensor_df, on='time', how='left')
/var/folders/z0/3x2n2w256hqc2mg2n4y7_0t00000gn/T/ipykernel_83505/4041771514.py:32: FutureWarning: Passing 'suffixes' which cause duplicate columns {'heart_rate/heart_rate_x'} in the result is deprecated and will raise a MergeError in a future version.
  match_df = match_df.merge(sensor_df, on='time', how='left')
